Project 1

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
#API use and web scrapping
import requests
from bs4 import BeautifulSoup
import re

In [2]:
sqlite_path = 'data/raw_data_project_m1.db'
conn_str = f'sqlite:///{sqlite_path}'
engine = create_engine(conn_str)

career_info 

In [3]:
query = 'SELECT * FROM career_info;'

In [4]:
career_info = pd.read_sql_query(query, engine)
career_info

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f
...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a


Normalized Job Titles

In [5]:
list_api = []
list_job_code = set(career_info['normalized_job_code'])

In [6]:
for i in list_job_code:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    result_jobs = response.json()
    list_api.append(result_jobs)
    
data_jobs = pd.DataFrame(list_api)


country_info

In [7]:
query2 = 'SELECT * FROM country_info;'
country_info = pd.read_sql_query(query2, engine)
country_info.head()

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city


personal_info

In [8]:
query3 = 'SELECT * FROM personal_info;'
personal_info = pd.read_sql_query(query3, engine)

poll_info

In [9]:
query4 = 'SELECT * FROM poll_info;'
poll_info = pd.read_sql_query(query4, engine)

API

In [10]:
response = requests.get('http://api.dataatwork.org/v1/jobs')

json_data = response.json()

In [11]:
job_info = pd.DataFrame(json_data)

Web sacrapping

In [12]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
country_codes = soup.find_all('td')

In [13]:
list_country_codes =[c.text.strip().split('\n') for c in country_codes]


In [14]:
list_country_codes2 = [[l for l in sub if len(l)>1] for sub in list_country_codes]
list_country_codes3 = [l for sub in list_country_codes2 for l in sub]

In [15]:
codes_raw = []
countries= []
for i in list_country_codes3:
    if len(i)<5 or i == '(XK[1])' or i == '(CN_X_HK)':
        codes_raw.append(i)
    else:
        countries.append(i)

In [16]:
codes_str = ''.join(codes_raw)
codes = re.sub(r'[^a-zA-Z ]', ' ', codes_str).split()

In [17]:
codes[57]= 'CN_X_HK'
del codes[58]
del codes[58]

MERGED DF

In [18]:
codes_df = pd.DataFrame(codes)
countries_df = pd.DataFrame(countries)
country_codes_info = pd.concat([codes_df, countries_df.reindex(codes_df.index)], axis=1)
country_codes_info.columns = ['country_code', 'country']
country_codes_info.head()

,country_code,country
0,BE,Belgium
1,EL,Greece
2,LT,Lithuania
3,PT,Portugal
4,BG,Bulgaria


In [19]:
df_merge_countries = pd.merge(country_codes_info,
                 country_info,
                 on='country_code')
df_merge_countries = df_merge_countries[['uuid','country_code','country', 'rural']]
df_merge_countries

,uuid,country_code,country,rural
0,dcab3aa0-dc79-0133-412b-0a81e8b09a82,BE,Belgium,urban
1,2fb4da70-dcef-0133-cafd-0a81e8b09a82,BE,Belgium,urban
2,73771070-d8ec-0133-ae53-0a81e8b09a82,BE,Belgium,urban
3,3aa979b0-d93d-0133-c00b-0a81e8b09a82,BE,Belgium,rural
4,11c6e300-deb8-0133-5b60-0a81e8b09a82,BE,Belgium,urban
...,...,...,...,...
8220,a1ceab50-da22-0133-ea30-0a81e8b09a82,PL,Poland,urban
8221,74fec3c0-db5b-0133-dab4-0a81e8b09a82,PL,Poland,urban
8222,f8dc4450-da13-0133-97bb-0a81e8b09a82,PL,Poland,Non-Rural
8223,91b54950-d911-0133-c958-0a81e8b09a82,PL,Poland,city


In [20]:
data_jobs.head()
 

,uuid,title,normalized_job_title,parent_uuid,error
0,847165cfda6b1dc82ae22b967da8af2f,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,NaN
1,970eb38c7b3eb080466052ea994af189,Internet Database Specialist,internet database specialist,a5773315d55245d7a3271314a564c32f,NaN
2,3ddea1b52e520996e4ec9fb70dc8b86b,Field Artillery Automated Tactical Data System...,field artillery automated tactical data system...,89d82eabfbe83755f4435bc7fcc8d6af,NaN
3,1c37a6e2bf157c0eff4665c57c52602d,SCADA Technician (Supervisory Control and Data...,scada technician supervisory control and data ...,5e618fbfb1206c29ad9b4c3e1d169ba5,NaN
4,122f24bc911a79c26ca66210557dda07,Clinical Data Specialist,clinical data specialist,db06efbaa9cb31d42e02c047ffb0a15a,NaN


In [21]:
career_info.columns

Index(['uuid', 'dem_education_level', 'dem_full_time_job',
       'normalized_job_code'],
      dtype='object')

In [22]:
df_merge_jobs = pd.merge(data_jobs, career_info, how= 'inner', left_on= ['uuid'], right_on = ['normalized_job_code'])


In [23]:
df_merge_jobs.head()

,uuid_x,title,normalized_job_title,parent_uuid,error,uuid_y,dem_education_level,dem_full_time_job,normalized_job_code
0,847165cfda6b1dc82ae22b967da8af2f,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,NaN,c76aa3e0-dbf6-0133-0d62-0a81e8b09a82,None,yes,847165cfda6b1dc82ae22b967da8af2f
1,847165cfda6b1dc82ae22b967da8af2f,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,NaN,3aa979b0-d93d-0133-c00b-0a81e8b09a82,medium,yes,847165cfda6b1dc82ae22b967da8af2f
2,847165cfda6b1dc82ae22b967da8af2f,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,NaN,67bd25c0-da23-0133-ed4d-0a81e8b09a82,medium,yes,847165cfda6b1dc82ae22b967da8af2f
3,847165cfda6b1dc82ae22b967da8af2f,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,NaN,ce9f80c0-d94a-0133-89ac-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f
4,847165cfda6b1dc82ae22b967da8af2f,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,NaN,164fa0c0-d975-0133-36e4-0a81e8b09a82,medium,yes,847165cfda6b1dc82ae22b967da8af2f


In [24]:
df_raw = pd.merge(df_merge_countries, df_merge_jobs, left_on = 'uuid', right_on = 'uuid_y')

In [25]:
df_raw.drop(['uuid_x', 'parent_uuid', 'error', 'uuid_y', 'dem_education_level', 'dem_full_time_job', 'normalized_job_title', 'normalized_job_code'], axis = 1, inplace = True)

In [26]:
df_raw

,uuid,country_code,country,rural,title
0,dcab3aa0-dc79-0133-412b-0a81e8b09a82,BE,Belgium,urban,NaN
1,2fb4da70-dcef-0133-cafd-0a81e8b09a82,BE,Belgium,urban,NaN
2,73771070-d8ec-0133-ae53-0a81e8b09a82,BE,Belgium,urban,Data Security Administrator
3,3aa979b0-d93d-0133-c00b-0a81e8b09a82,BE,Belgium,rural,Data Warehouse Developer
4,11c6e300-deb8-0133-5b60-0a81e8b09a82,BE,Belgium,urban,Geographic Information Systems Data Manager (G...
...,...,...,...,...,...
8220,a1ceab50-da22-0133-ea30-0a81e8b09a82,PL,Poland,urban,Data Storage Specialist
8221,74fec3c0-db5b-0133-dab4-0a81e8b09a82,PL,Poland,urban,Automatic Data Processing Planner
8222,f8dc4450-da13-0133-97bb-0a81e8b09a82,PL,Poland,Non-Rural,NaN
8223,91b54950-d911-0133-c958-0a81e8b09a82,PL,Poland,city,NaN


In [36]:
quantity = df_raw.groupby(['country','title','rural']).agg({'uuid':['count']})
percentage = (quantity/quantity).sum()*100

In [37]:
df_raw['quantity'] = quantity

TypeError: incompatible index of inserted column with frame index

In [38]:
df_raw['percentage'] = percentage

TypeError: incompatible index of inserted column with frame index